# Java CLI

In [18]:
JAVABIN = "java -jar ../javallier/target/scala-2.10/javallier.jar"
PYBIN = "~/py35/bin/pheutil"

print("Check we can access both command line tools:")
!{PYBIN} --version
!{JAVABIN}

Check we can access both command line tools:
pheutil, version 1.0-alpha

usage: javallier COMMAND [OPTIONS]
Javallier CLI - Data61 - 2016
Commands:
    add: Add ENCRYPTED to PLAINTEXT
    genpkey: Create a new paillier keypair
    extract: Extract the public key from a PRIVATE key
    encrypt: Encrypt a value with the given public key
    decrypt: Decrypt CIPHERTEXT with PRIVATEKEY
    multiply: Multiply ENCRYPTED with PLAINTEXT
    addenc: Add ENCRYPTED1 to ENCRYPTED2
Try javallier COMMAND --help for command usage.

Options:

 -h,--help      Show this message and exit.
 -v,--verbose   Enable logging

Please report issues to https://github.com/NICTA/javallier/issues


In [3]:
test_numbers = [0, 1, 1000, 5000, 1e12]

## Test 1 - Encryt with key generated from Python

In [4]:
# Generate the key using Python
!{PYBIN} genpkey --keysize 1024 py_1024_priv.json
!{PYBIN} extract py_1024_priv.json py_1024_public.json

Generating a paillier keypair with keysize of 1024
Keys generated
Private key written to py_1024_priv.json
Loading paillier keypair
Public key written to py_1024_public.json


In [8]:
# Encrypt using both Python and Java
for i, num in enumerate(test_numbers[:1]):
    #!{PYBIN} encrypt --output py_{i}.enc py_1024_public.json {num}
    !{JAVABIN} encrypt --output java_{i}.enc py_1024_public.json {num}


Feb 02, 2017 2:07:35 PM com.n1analytics.paillier.cli.JavallierCLI parse
com.n1analytics.paillier.EncodeException: Input value cannot be encoded.
	at com.n1analytics.paillier.StandardEncodingScheme.innerEncode(StandardEncodingScheme.java:293)
	at com.n1analytics.paillier.StandardEncodingScheme.encode(StandardEncodingScheme.java:139)
	at com.n1analytics.paillier.PaillierContext.encode(PaillierContext.java:295)
	at com.n1analytics.paillier.PaillierContext.encrypt(PaillierContext.java:491)
	at com.n1analytics.paillier.cli.JavallierCLI$EncryptCommand.run(JavallierCLI.java:360)
	at com.n1analytics.paillier.cli.JavallierCLI.parse(JavallierCLI.java:112)
	at com.n1analytics.paillier.cli.Main.main(Main.java:9)


In [14]:
import json
with open('py_1024_public.json') as f:
    print("Python public key encoding:")
    print(json.load(f)['n'])
    
print("\nJava public key encoding:")
with open('java_1024_public.json') as f:
    print(json.load(f)['n'])

Python public key encoding:
slI-NT0mzQfAegnIgDjV4Ckt5EIjDFWF41jY3B3dLETIfN-ZMyT4sWZHNXN0XJmlQQIcaoy_UoSZvwBVTYswePLsvCLf-vDwB3HfDDe0WzySYGxxA1BIKUbUYgnNTVfXUYV11TzROFs96uBpkTgZvycYKaALancet5AmzJPObg0

Java public key encoding:
AIkw19cGIHcFFH/bg2xnN5YvBkfwpPDfDhWg6LEZxkWYeaZaKsC/n0Z8+AEI2gy4fIMq6M3ZOO6kge02DRxUIHvavaqNB4bvV3A3svRpADwKV98lfV/OodLNysAkGICe9kMB77xhdwBKZWCr1VxhxBeHsGleeg4MpXAL9J8gKuUP


As we can see the Python encoding conforms to RFC7517, but the Java encoding does not.

In [28]:
for i, num in enumerate(test_numbers):
    !{BIN} decrypt --output py_{i}.plaintext py_1024_priv.json py_{i}.enc 
    
    out = open("py_{}.plaintext".format(i)).read().strip()
    print(out)
    print(float(num))

Loading private key
Decrypting ciphertext
0.0
0.0
Loading private key
Decrypting ciphertext
1.0
1.0
Loading private key
Decrypting ciphertext
1000.0
1000.0
Loading private key
Decrypting ciphertext
5000.0
5000.0
Loading private key
Decrypting ciphertext
1000000000000.0
1000000000000.0


In [29]:



for i, num in enumerate(test_numbers):
    !{BIN} decrypt --output py_{i}.plaintext py_1024_priv.json py_{i}.enc 
    
    out = open("py_{}.plaintext".format(i)).read().strip()
    print(out)
    print(float(num))


Running the decrypt command
[decrypt, py_1024_priv.json, py_0.enc]
Feb 02, 2017 11:35:29 AM com.n1analytics.paillier.cli.JavallierCLI parse
java.lang.IllegalArgumentException: totient must be non-negative
	at com.n1analytics.paillier.PaillierPrivateKey.<init>(PaillierPrivateKey.java:156)
	at com.n1analytics.paillier.cli.SerialisationUtil.unserialise_private(SerialisationUtil.java:34)
	at com.n1analytics.paillier.cli.JavallierCLI$DecryptCommand.run(JavallierCLI.java:436)
	at com.n1analytics.paillier.cli.JavallierCLI.parse(JavallierCLI.java:112)
	at com.n1analytics.paillier.cli.Main.main(Main.java:9)

0.0

Running the decrypt command
[decrypt, py_1024_priv.json, py_1.enc]
Feb 02, 2017 11:35:29 AM com.n1analytics.paillier.cli.JavallierCLI parse
java.lang.IllegalArgumentException: totient must be non-negative
	at com.n1analytics.paillier.PaillierPrivateKey.<init>(PaillierPrivateKey.java:156)
	at com.n1analytics.paillier.cli.SerialisationUtil.unserialise_private(SerialisationUtil.java:34)
